# PyTorch introduction: Tensors (Chapter 3)

# Constructing our first tensor

In [1]:
import torch

#### We will create a one dimensional tensor of size 3 filled with 1s.

In [2]:
a = torch.ones(3)
a

tensor([1., 1., 1.])

In [3]:
a[1]

tensor(1.)

In [4]:
float(a[1])

1.0

In [5]:
a[2] = 2.0
a

tensor([1., 1., 2.])

#### Now let's store a list of coordinates of a triangle. We will store Xs in the even and Ys in the odd.

In [6]:
points = torch.zeros(6)

points[0] = 4.0
points[1] = 1.0
points[2] = 5.0
points[3] = 3.0
points[4] = 2.0
points[5] = 1.0

#### We can also do it like this

In [7]:
points = torch.tensor([4.0, 1.0, 5.0, 3.0, 2.0, 1.0])
points

tensor([4., 1., 5., 3., 2., 1.])

#### Coordinates of the first points:

In [8]:
float(points[0]), float(points[1])

(4.0, 1.0)

#### It's better to use a 2D tensor for this purpose

In [9]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

#### Let's ask for it's shape

In [10]:
points.shape

torch.Size([3, 2])

#### We can create a tensor with the desired shape

In [11]:
points = torch.zeros(3,2)
points

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])

#### We can access an individual element using two indices. We will return the Y-coord of the first element.

In [12]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])

points[0,1]

tensor(1.)

#### We can acces multiple elements and choosing which of them we want

In [13]:
points[1:] # All rows after the first, implicitly all columns
points[1:, :] # All rows after the first; all columns
points[1:, 0] # All rows after the first; first column
points[None] # Adds a dimension of size 1, just like unsqueeze

tensor([[[4., 1.],
         [5., 3.],
         [2., 1.]]])

## Named Tensors

#### We want to turn an image into gray scale

In [14]:
img_t = torch.randn(3, 5, 5) # shape [channels, rows, columns]
weights = torch.tensor([0.2126, 0.7152, 0.0722])

#### We also can generalize images adding third channel dimensions (as in RGB) or from a single image to a batch of them.

In [15]:
batch_t = torch.randn(2, 3, 5, 5) #shape [batch, channels, rows, columns]

#### And we can eliminate channels to turn into gray

In [16]:
img_gray_naive = img_t.mean(-3) # Eliminating RGB (-3 is the first, third counting from the end)
batch_gray_naive = batch_t.mean(-3) # Eliminating RGB
img_gray_naive.shape, batch_gray_naive.shape

(torch.Size([5, 5]), torch.Size([2, 5, 5]))

#### But now we have the weight, too. We can multiply things that are the same shape, as well as shapes where one operand is of size 1 in a given dimension. It also appends leading dimensions of size 1 automatically. 

#### batch_t of shape (2, 3, 5, 5) is multiplied by unsqueezed_weights of shape (3, 1, 1), resulting in a tensor of shape (2, 3, 5, 5), from which we can then sum the third dimension from the end (the three channels).

In [17]:
unsqueezed_weights = weights.unsqueeze(-1).unsqueeze_(-1)

img_weights = (img_t * unsqueezed_weights)
batch_weights = (batch_t * unsqueezed_weights)

img_gray_weighted = img_weights.sum(-3)
batch_gray_weighted = batch_weights.sum(-3)

batch_weights.shape, batch_t.shape, unsqueezed_weights.shape

(torch.Size([2, 3, 5, 5]), torch.Size([2, 3, 5, 5]), torch.Size([3, 1, 1]))

### Tensor value types

#### We can change the value types a tensor handles with the attribute dtype. Most common ones are 32-bit float and 64-bit int.

In [18]:
double_points = torch.ones(10, 2, dtype = torch.double)
short_points = torch.tensor([[1, 2], [3, 4]], dtype = torch.short)

double_points = torch.zeros(10, 2).to(torch.double)
double_points = torch.zeros(10, 2).to(dtype = torch.short)

## The Tensor API

In [19]:
a = torch.ones(3,2)
a_t = torch.transpose(a, 0, 1) # Can be also done with a_t = a.transpose(0, 1)

a.shape, a_t.shape

(torch.Size([3, 2]), torch.Size([2, 3]))

### Tensor operations

In [20]:
create = torch.ones(10,2) # Operation for creating tensors (ones, zeros... etc)
transpose = torch.transpose(create, 0, 1) # Shape-changing functions
# Math ops (pointwise, reduction, comparaison, spectral, other...)
rand = torch.randn(10,2) # Random drawing from distributions

## Tensor storages: Management

### Accessing a tensor's storage

In [21]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points.storage()

 4.0
 1.0
 5.0
 3.0
 2.0
 1.0
[torch.FloatStorage of size 6]

#### We can also index into a storage manually as in this example below

In [22]:
points_storage = points.storage()

points_storage[0], points.storage()[1]

(4.0, 1.0)

### Understanding metadata: Offset, Stride, Shape

#### The size or shape is a tuple indicating how many elements across each dimension the tensor represents. The storage offset is the index in the storage corresponding to the first element in the tensor. The stride is the number of elements in the storage that need to be skipped over to obtain the next element along each dimension.

#### We can access de 2nd index of a tensor by providing the index:

In [23]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
second_point = points[1]
second_point.storage_offset()

2

In [24]:
second_point.size()

torch.Size([2])

In [25]:
points.stride()

(2, 1)

#### Accessing an element i, j in a 2D tensor results in accessing the storage_offset + stride[0] * i + stride[1] * j element in the storage. The offset will usually be zero; if this tensor is a view of a storage created to hold a larger tensor, the offset might be a positive value.

## How subtensors work

#### If we change the subtensor, it will affect the principal tensor

In [26]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])

second_point = points[1]
second_point[0] = 10.0

points

tensor([[ 4.,  1.],
        [10.,  3.],
        [ 2.,  1.]])

#### If this is not our objective we can just clone the subtensor instead of linking it

In [27]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])

second_point = points[1].clone()
second_point[0] = 10.0

points

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

## Transposing tensors

In [28]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

In [29]:
points_t = points.t()
points_t

tensor([[4., 5., 2.],
        [1., 3., 1.]])

#### We can prove that both tensors have the same storage, but they differ in shape and stride

In [30]:
id(points.storage()) == id(points_t.storage())

True

In [31]:
points.stride()

(2, 1)

In [32]:
points_t.stride()

(1, 2)

### Transposing multidimensional tensors

#### We can specify the two dimensions to transpose

In [33]:
some_t = torch.ones(3, 4, 5)
transpose_t = some_t.transpose(0, 2)
some_t.shape

torch.Size([3, 4, 5])

In [34]:
transpose_t.shape

torch.Size([5, 4, 3])

In [35]:
some_t.stride()

(20, 5, 1)

In [36]:
transpose_t.stride()

(1, 5, 20)

## Serializing and saving Tensors as Pickle

### Saving weights: Two options

In [37]:
# torch.save(points, '../data/p1ch3/ourpoints.t')

#  with open('../data/p1ch3/ourpoints.t','wb') as f:
#   torch.save(points, f)

### Loading weights: Two options

In [38]:
# points = torch.load('../data/p1ch3/ourpoints.t')

#   with open('../data/p1ch3/ourpoints.t','rb') as f:
#    points = torch.load(f)

## Serializing to HDF5: Storing as NumPy arrays